# Preprocesamiento para Experimentos de altísima frecuencia y grandes conjuntos de datos
 
Este Cuaderno es muy similar al Preprocesamiento para Evocados, pero sin promediar sobre múltiples estímulos. 
Dado que estos experimentos son muy grandes y causan problemas con el RAM, lo que vamos a hacer es primero cortarlos en cachos. En este cuaderno asumimos que ya se cortó o no fue necesario hacerlo. Para cortar revisar el programa CortarMatriz.jl en la carpeta Isabel.

In [1]:
push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../Isabel/")
using Z_auxiliaresBRW
using PreprocUInt8
using HDF5, PyPlot, Statistics
using ArraySetTools

┌ Info: Recompiling stale cache file /home/karel/.julia/compiled/v1.0/PreprocUInt8.ji for PreprocUInt8 [top-level]
└ @ Base loading.jl:1190


In [2]:
arxname="/home/karel/BRWFiles/Isabel2019/Cacho_04_control_02.brw" 

"/home/karel/BRWFiles/Isabel2019/Cacho_04_control_02.brw"

Vamos a hacer que pueda abrir tanto un brw original como un cacho arreglado o manipulado.
Tratemos de que funcione con los nuevos unicamente. Luego vemos si generalizamos.

In [3]:
if occursin("Cacho", arxname)
    arx=h5open(arxname)
    nota=read(arx["notacacho"])
    print(nota)
else
    arx=brw_things(arxname)
    print("Estas partiendo de un brw original")
end

freq=read(arx["SamplingRate"])/1000 # usamos kHz y milisegundos aqui porfa
dataraw=read(arx["dset"])
factor=read(arx["Factor"])
listchans=read(arx["Chs"])
evocado=false

Este es un intervalo de un dataset mas grande. 

                    Contiene los cuadros de 1205245 a 1606992.

false

In [4]:
tamanos=size(dataraw)
if length(tamanos)==2
    ncuadros=tamanos[2]
else
    tot=tamanos[1]
    ncuadros=Int(tamanos/4096)
end

401748

In [5]:
#Estas medidas estan en cuadros, no en ms.
if evocado
    retrazo=round(Int, ceil(5*freq))
    final=round(Int, ceil(60*freq))
    latencia=round(Int, ceil(1.5*freq))
else
    retrazo, final, latencia = 0, 0, 0
end

(0, 0, 0)

In [6]:
tiempototalms=round(ncuadros/freq; digits=1) 
println("Tienes ", ncuadros, " cuadros de muestreo a ", round(freq; digits=4), " cuadros por milisegundo")
println( "Esto corresponde a  ", tiempototalms, "ms." )

Tienes 401748 cuadros de muestreo a 17.8555 cuadros por milisegundo
Esto corresponde a  22500.0ms.


In [7]:
# Si los datos andan en un arreglo de lista en lugar de cuadrado, los ponemos cuadrados
if size(dataraw)[1] != 4096
reshape!(dataraw, (4096, Datos["numcuadros"]))
end;

In [33]:

function muvoltint(x, factor, offset)
    result=x/factor-offset
end


function bsht(datos::Array;
                         factor,
                         offset=-2048,
                         tini=0.5,
                              tfin=8, freq=deffreq,
                              maxvolt=-100, minvolt=-1500, 
                              minstd=10, maxstd=35)
    #Busquemos los canales con probable respuesta de potencial de accion
    (nchans,nc)=size(datos)
    taux1=round(Int, ceil(tini*freq))
    taux2=round(Int,ceil(tfin*freq))

    mxvint=muvoltint(maxvolt, factor, offset)
    mnvint=muvoltint(minvolt, factor, offset)
    ustdint=abs(maxstd/factor)
    lstdint=abs(minstd/factor)
    
    println("Estoy buscando del cuadro " , taux1, " al , ", taux2)

    result=Set(Array{Int8, 1}[])
    
    for j=1:nchans
        
        fondo=minimum(vec(datos[j,taux1:taux2]))
        dpgs=std(datos[j,taux1:taux2])
        test=true
        
        if factor>0
            test=(mxvint >fondo>mnvint) && ( ustdint > dpgs > lstdint)
        else
            test=(mxvint <fondo<mnvint) && ( ustdint > dpgs > lstdint)
        end
           
        if test
            println("EL MUGRE IF SE EST CUIMPLUIENDO")       
            (b,a)=divrem(j, 64)
            println(a, b)       
        
            #los canales se cuentas desde 1, no de cero
            b+=1
            a+=1
            println(a, b)       
        
            bla= [j, a, b]
            println(bla, a, b)       
            result=push!(result,bla)
        end
    end

    #= Esta rutina no es muy confiable. Solo da buenos resultados
    con actividad evocada. Necesitamos algo mas estricto =#
   
    return result
end


bsht (generic function with 1 method)

In [37]:
8000/65000

0.12307692307692308

In [34]:
#los parametros son datos promediados, frecuencia, tiempo post estimulo en ms, umbral en microvolts, umbral de saturación en microvolts

iniciobusqueda=1
finbusqueda=282

PruebaRespuesta=bsht(dataraw, factor=factor,
    tini=iniciobusqueda, tfin=finbusqueda, freq=freq)
# y luego los saturados

#=
satu=BuscaSaturados(lfp,iniciobusqueda,finbusqueda,freq,1000, 0.04)

numerosaturados=length(satu)
println("Encontramos ", numerosaturados, " canales probablemente saturados.")
println("Encontramos ", length(PruebaRespuesta), " canales probablemente con actividad.")
=#

Estoy buscando del cuadro 18 al , 5036
EL MUGRE IF SE EST CUIMPLUIENDO
390
401
[39, 40, 1]401
EL MUGRE IF SE EST CUIMPLUIENDO
450
461
[45, 46, 1]461
EL MUGRE IF SE EST CUIMPLUIENDO
182
193
[146, 19, 3]193


InexactError: InexactError: trunc(Int8, 146)

In [ ]:
quietos=BuscaSaturadosStd(lfp, iniciobusqueda,finbusqueda, freq, 15,10000)
intensos=BuscaSaturadosStd(lfp, iniciobusqueda, finbusqueda, freq, 0, 30)

numerosaturados=length(quietos)
println("Encontramos ", numerosaturados, " canales muy quietos")
numerosaturados=length(intensos)
println("Encontramos ", numerosaturados, " canales muy intensos")

ruidosos=BuscaRuidosos(lfp, iniciobusqueda, finbusqueda, freq,200,15)
numerosaturados=length(ruidosos)
println("Encontramos ", numerosaturados, " canales con ruido alto")

### inspección visual. 
Podemos tomar una imagen de la desviación estándar por canal y sobreponerle unas marcas para los canales "respuesta".
De ahí podemos tomar un ejemplo para dibujar el trazo en ese canal.

In [ ]:
DesviacionPorCanal=zeros(64,64)
for j=1:64
    for k=1:64
        ChorizoExemplo=vec(lfp[j,k,:])
        DesviacionPorCanal[j,k]=std(ChorizoExemplo)
    end
end

In [ ]:
figure(figsize=(10,10))
xlim(0,65)
ylim(0,65)
title("σ")
limites=90
imagen=imshow(DesviacionPorCanal, origin="lower", interpolation="nearest",cmap="gnuplot2", 
                      vmin=10,vmax=limites, extent=[0.5,64.5,0.5,64.5])
cb=colorbar()

x=[]
y=[]
for j in PruebaRespuesta
  append!(x,j[2])
   append!(y,j[1])
end


x2=[]
y2=[]
for j in satu
  append!(x2,j[2])
   append!(y2,j[1])
end


x3=[]
y3=[]
for j in quietos
  append!(x3,j[2])
   append!(y3,j[1])
end


x4=[]
y4=[]
for j in intensos
  append!(x4,j[2])
   append!(y4,j[1])
end


x5=[]
y5=[]
for j in ruidosos
  append!(x5,j[2])
   append!(y5,j[1])
end





scatter(x,y, marker="o",c="lightblue", s=15)
scatter(x2,y2, marker="o",c="green", s=5)
scatter(x3,y3, marker="x",c="green", s=25)
scatter(x4,y4, marker="+",c="lightblue", s=25)
scatter(x5,y5, marker="v",c="green", s=25)



"ruidosos" parece ser totalmente redundante. No aporta nada que no sea JUSTO lo que no nos interesa: la union de lo peor y lo mejor.
Escogamos uno de los que parecen estar en la "sombra luminosa" del Cuerno de Amon y que estén marcados con x blanca.

In [ ]:
egtrazo=lfp[19,42,:];

In [ ]:
using Statistics

In [ ]:
mean(egtrazo)

In [ ]:
figure(figsize=(12,4))
xlabel("cuadros")
ylabel("mV")
ylim(-180,180)
#subintervalo a graficar, por omision, todo primero, es decir desde=1, hasta=cuadrosmax
desde=1
hasta=8000
#intervalo=1:cacho
intervalo=desde:hasta
grid()
plot(intervalo,egtrazo[intervalo], lw=0.22)

In [ ]:
mal=union(quietos, satu)
buenos=union(intensos, PruebaRespuesta)
setdiff!(buenos, mal)

In [ ]:
malforsave=elemtorow(mal)
buenforsave=elemtorow(buenos);

In [ ]:
size(lfp)

In [ ]:
names(arx)

In [ ]:
basename(arxname)[1:end-3]

Tomaremos el intervalo interesante y guardaremos esos datos en un archivo con toda la información de saturados y respuestas, 
de la misma forma que como se hace con los evocados. A diferencia de éstos, no tenemos parámetros de latencia, en lugar de eso
tendremos una indicacion de "de donde a donde se cortaron los datos", en cuadros.

In [ ]:
dirgen=dirname(arxname)*"/"
basegen=basename(arxname)[1:end-4]
#una palabra para indicar el intervalo 
outname=string(dirgen, basegen, "_preproc.h5")
#aqui hay que arreglar el rollo del directorio

listaaguardar=Dict(
    "LFPSaturados" => lfp[:,:,:],
     "notacacho"=>nota,
     "freq" =>freq,
     "CanalesBuenos" => buenforsave,
     "CanalesMalos" => malforsave)

println(outname )
   h5open(outname, "w")  do file
        for k in keys(listaaguardar)
            datos=listaaguardar[k]
            println("abriendo los datos ", k)
            println
            write(file, "$k", datos)
        end
    end

println("hemos guardado este cacho en el archivo ", outname)

In [ ]:
typeof(listaaguardar)

In [ ]:
freq